In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installations

In [2]:
!pip install keras


In [3]:
! pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.2 MB/s eta 0:00:00


## Imports

In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer

from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

import gensim
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings(action = 'ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
#Preprocessing function
def preprocessing(data_frame):
    ## Preprocessing
    # Removing URLs whithin the tweets
    data_frame["Text"] = data_frame["Text"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
    # Removing emails, hashtags and punctuations
    data_frame['Text'] = data_frame["Text"].str.replace(r'\S*@\S*\s?', ' ').str.strip()
    data_frame['Text'] = data_frame['Text'].str.replace(r'#\S*\s?', ' ').str.strip()
    data_frame['Text'] = data_frame['Text'].str.replace(r'[^\w\s]+', ' ').str.strip()

    # Lowercase Text
    data_frame['Text'] = data_frame['Text'].str.lower()

    # # Removing stopwords
    stop = stopwords.words('english')
    data_frame['Text'].apply(lambda x: [item for item in str(x) if item not in stop])

    # Removing newline characters
    data_frame['Text'] = data_frame['Text'].str.rstrip()

    # Tokenizing Posts and counting the length of each post
    data_frame['Tokens'] = data_frame.apply(lambda row: word_tokenize(str(row['Text'])), axis=1)
    data_frame['Length'] = data_frame.apply(lambda row: len(row['Tokens']), axis=1)

    return data_frame

# Data

In [ ]:
DS_Path = "./Datasets"

### Twitter 1

In [ ]:
# ## Preparing the data
# ## Twitter 10000
Twitter_path = DS_Path + "/Twitter/twitter-suicidal_data_10000.csv"
df = pd.read_csv(Twitter_path, encoding='latin-1')
df = df.rename(columns={'tweet': 'Text', 'intention': 'Label'})
df = preprocessing(df)
df


,Text,Label,Tokens,Length
0,my life is meaningless i just want to end my l...,1,"[my, life, is, meaningless, i, just, want, to,...",79
1,muttering i wanna die to myself daily for a fe...,1,"[muttering, i, wan, na, die, to, myself, daily...",46
2,work slave i really feel like my only purpose ...,1,"[work, slave, i, really, feel, like, my, only,...",69
3,i did something on the 2 of october i overdose...,1,"[i, did, something, on, the, 2, of, october, i...",77
4,i feel like no one cares i just want to die ma...,1,"[i, feel, like, no, one, cares, i, just, want,...",18
...,...,...,...,...
9114,have you ever laid on your bed at night and cr...,1,"[have, you, ever, laid, on, your, bed, at, nig...",33
9115,the fault the blame the pain s still there i m...,1,"[the, fault, the, blame, the, pain, s, still, ...",20
9116,stop asking me to trust you when i m still cou...,1,"[stop, asking, me, to, trust, you, when, i, m,...",22
9117,i never know how to handle sadness crying make...,1,"[i, never, know, how, to, handle, sadness, cry...",12


### Twitter 2

In [ ]:
twitter_path = DS_Path + "/suicidal-tendency-tweets.csv"
df = pd.read_csv(twitter_path, encoding='latin-1', usecols=['tweet', 'intention'], nrows = 17142)
df = df.rename(columns={'tweet': 'Text', 'intention': 'Label'})

# df = preprocessing(df)
# df

In [ ]:
print(list(df['Label']).count(1), list(df['Label']).count(0))

3754 13388


In [ ]:
difference = np.abs(list(df['Label']).count(0)-list(df['Label']).count(1))
difference

9634

In [ ]:
Path_Twitter_56000 = DS_Path + "/Twitter/suicideTweetData_56000.csv"
twitter_1_samples= pd.read_csv(Path_Twitter_56000, nrows=difference)
twitter_1_samples
ones = twitter_1_samples[twitter_1_samples['label'] == -1]
ones = ones.replace(-1, 1)
ones = ones.rename(columns={'Tweet': 'Text', 'label': 'Label'})
df = pd.concat([df, ones])
df

,Text,Label
0,@ModernTosspot @kurtistrippled @Jebadoo2 to go...,1
1,@ModernTosspot @kurtistrippled @Jebadoo2 Thy ...,1
2,@notsmokinfeds @DirtyDianaDyke @LiberatedLezzy...,0
3,@notsmokinfeds @DirtyDianaDyke @LiberatedLezzy...,0
4,@Nikki40_40 @AJLIESHERE @littlezupa @kurtistri...,0
...,...,...
9629,its been a long time since i wanted to die but...,1
9630,i desperately wish i could take control of my ...,1
9631,i think i need swimming lessons cause i’m dro...,1
9632,i mean i still want to kill myself but whateva,1


In [ ]:
print(list(df['Label']).count(1), list(df['Label']).count(0))

13388 13388


In [ ]:
df = preprocessing(df)

In [ ]:
i=2

## Reddit SNS

In [6]:
# ## Preparing the data
## Loading Raw Data
Reddit_path = DS_Path + "/Reddit_SNS/Suicide_Detection.csv"

# i = 1
# df_r = pd.read_csv(Reddit_path, encoding='latin-1', usecols=['text', 'class'])[(i-1)*10000: i*10000]
df_r = pd.read_csv(Reddit_path, encoding='latin-1', usecols=['text', 'class'], nrows = 20000)
df_r = df_r.rename(columns={'text': 'Text', 'class': 'Label'})

df = preprocessing(df_r)

label_dict = {'suicide': 1, 'non-suicide': 0}
df['Label'] = df['Label'].apply(lambda row: label_dict[row])
df['Label']

df

,Text,Label,Tokens,Length
0,ex wife threatening suiciderecently i left my ...,1,"[ex, wife, threatening, suiciderecently, i, le...",147
1,am i weird i don t get affected by compliments...,0,"[am, i, weird, i, don, t, get, affected, by, c...",29
2,finally 2020 is almost over so i can never he...,0,"[finally, 2020, is, almost, over, so, i, can, ...",27
3,i need helpjust help me im crying so hard,1,"[i, need, helpjust, help, me, im, crying, so, ...",9
4,iâ m so losthello my name is adam 16 and iâ...,1,"[iâ, m, so, losthello, my, name, is, adam, 16,...",459
...,...,...,...,...
19995,i just wish i wasn t like thisi read a lot of ...,1,"[i, just, wish, i, wasn, t, like, thisi, read,...",523
19996,future druggie why do i see the only reason to...,1,"[future, druggie, why, do, i, see, the, only, ...",62
19997,i just tried to kill myself help i just tried...,1,"[i, just, tried, to, kill, myself, help, i, ju...",44
19998,someone downvoted my post about me being a pie...,1,"[someone, downvoted, my, post, about, me, bein...",47


In [7]:
print(list(df['Label']).count(1), list(df['Label']).count(0))

9865 10135


### Reddit SNS Random Selection

In [ ]:
Reddit_path = DS_Path + "/Reddit_non suicide  suicide/Suicide_Detection.csv"

df_r = pd.read_csv(Reddit_path, encoding='latin-1', usecols=['text', 'class'])
df_r = df_r.rename(columns={'text': 'Text', 'class': 'Label'})
label_dict = {'suicide': 1, 'non-suicide': 0}
df_r['Label'] = df_r['Label'].apply(lambda row: label_dict[row])

n_sample = 10000
suicidal_posts = df_r[df_r['Label']==1]
nonsuicidal_posts = df_r[df_r['Label']==0]

s_df = suicidal_posts.sample(n=int(n_sample/2), replace=False , random_state=1)
ns_df = nonsuicidal_posts.sample(n=int(n_sample/2), replace=False, random_state=1)

print(list(s_df['Label']).count(1), list(ns_df['Label']).count(0))

5000 5000


In [ ]:
df = pd.concat([s_df, ns_df], ignore_index=True)

df = df.sample(frac = 1) # to shuffle the data

In [ ]:
print(list(df['Label']).count(1), list(df['Label']).count(0))
df

5000 5000


,Text,Label,Tokens,Length
9385,this subreddit is so much better at night idk ...,0,"[this, subreddit, is, so, much, better, at, ni...",23
4082,help i need help but donâ t want it iâ m only ...,1,"[help, i, need, help, but, donâ, t, want, it, ...",97
7726,i want cute girl \n\ni want girl i want cute g...,0,"[i, want, cute, girl, i, want, girl, i, want, ...",26
5437,day 1 of mocking people who mock others \nif...,0,"[day, 1, of, mocking, people, who, mock, other...",13
5625,â i do believe in fairies â heheheheheheheheh...,0,"[â, i, do, believe, in, fairies, â, hehehehehe...",13
...,...,...,...,...
7300,ik this is probably said way too much but skir...,0,"[ik, this, is, probably, said, way, too, much,...",17
8642,i had like 2 shots of rum again gt and now i ...,0,"[i, had, like, 2, shots, of, rum, again, gt, a...",28
6740,throwaway acc so imma vent i feel terrible i ...,0,"[throwaway, acc, so, imma, vent, i, feel, terr...",140
5816,holy shit my channel just passed 700 views wtf...,0,"[holy, shit, my, channel, just, passed, 700, v...",34


In [ ]:
df = preprocessing(df)

In [ ]:
print(list(s_df['Label']).count(1), list(ns_df['Label']).count(0))

5000 5000


# BERT Model

## Installations

In [8]:
# A dependency of the preprocessing for BERT inputs
!pip install -U "tensorflow-text==2.13.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 44.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.4:
      Successfully uninstalled gast-0.5.4
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
   

In [ ]:
!pip install "tf-models-official==2.13.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00


In [ ]:
# !pip install keras-tcn

!pip install keras-tcn --no-dependencies  # without the dependencies if you already have TF/Numpy.

## Main Model

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [ ]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [ ]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
# a custome Attention layer
class Attention(Layer):
  def __init__ (self, return_sequences=True):
    self.return_sequences = return_sequences
    super(Attention, self).__init__()

  def build (self, input_shape):
    self.W = self.add_weight(name="att_weight", shape=(input_shape[-1],1), initializer="normal")
    self.b = self.add_weight(name="att_bias", shape=(input_shape[1],1), initializer="zeros")
    super(Attention, self).build(input_shape)

  def call(self, x):
    e = K.tanh(K.dot(x, self.W)+self.b)
    a = K.softmax(e, axis=1)
    output = x*a
    if self.return_sequences:
      return output
    return K.sum(output, axis = 1)


In [ ]:
from tcn import TCN, tcn_full_summary
from keras.layers import Bidirectional
from tensorflow.keras.initializers import RandomNormal, Constant

from tensorflow.keras.layers import Bidirectional, Dense, Input, Embedding, Conv1D, GlobalMaxPooling1D, Dropout, LSTM
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras import regularizers
from tensorflow.keras.models import *

def tcn_model(kernel_size = 3, activation='relu'):

    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']

    x = tf.expand_dims(net, axis=1)  # Add a dummy dimension for timesteps

    x = SpatialDropout1D(0.1)(x)
    # x = LSTM(100, stateful=False, return_sequences = True)(x)
    x = CuDNNLSTM(100, stateful=False, return_sequences = True)(x)

    x = Bidirectional(TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1'))(x)
    x = Bidirectional(TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2'))(x)
    x = Attention(return_sequences=True)(x)

    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)

    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)

    output = Dense(1, activation="sigmoid")(conc)
    # output = Dense(2, activation="softmax")(conc)

    model = Model(inputs=text_input, outputs=output)
    return model

# model_tcn = tcn_model(kernel_size = 8, activation='relu')

In [ ]:
import ast
df['Tokens'] = df['Tokens'].apply(lambda x: ast.literal_eval[x] if isinstance(x, str) else x)

texts = []
for row in df['Tokens']:
  text = ""
  for item in row:
    text += " "+item
  texts.append(text)

texts = pd.DataFrame(texts)

In [ ]:
## Spliting the data
X, X_test, y, y_test = train_test_split(texts, df['Label'], shuffle=True, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=33)

print("Train = ", X_train.shape, y_train.shape)
print("Test = ", X_test.shape, y_test.shape)
print("Validation = ",X_val.shape, y_val.shape)

Train =  (12800, 1) (12800,)
Test =  (4000, 1) (4000,)
Validation =  (3200, 1) (3200,)


In [ ]:
import tensorflow as tf

# Define your input shape based on your data
# input_shape = (max_length, )  # Replace with your actual input shape
num_filters = 64  # Adjust the number of filters as needed

# Define MirroredStrategy for data parallelism
strategy = tf.distribute.MirroredStrategy()

# Create and compile your TCN model within the strategy scope
with strategy.scope():
    model_tcn = tcn_model(kernel_size = 8, activation='relu')
    model_tcn.compile( loss = 'binary_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])


# Define batch size and number of epochs
batch_size = 64
epochs = 30

# Create TensorFlow datasets from your training and validation data
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)

# Define callbacks or other configuration options as needed

# Training with data parallelism
history = model_tcn.fit(train_dataset, epochs=epochs, validation_data=val_dataset)


### Prediction

In [ ]:
#Prediction on the test data
pred = model_tcn.predict(X_test)
print(pred)

In [ ]:
y_pred = np.zeros(len(pred))
for item in range(len(pred)):
  # y_pred[item] = list(pred[item]).index(max(pred[item]))
  if min(pred[item]) >= 0.5:
    y_pred[item] = 1
  else:
    y_pred[item] = 0

print(y_pred)

In [ ]:
import sklearn

report = sklearn.metrics.classification_report(y_test, y_pred)
print(report)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred)
recall = sklearn.metrics.recall_score(y_test, y_pred)
f1score = sklearn.metrics.f1_score(y_test, y_pred)
print("-----------------------")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score", f1score)

## Saving the results
from pathlib import Path
results = {
    "Index": i,
    "Accuracy": accuracy,
    "Precision": precision,
    "Recall": recall,
    "F1score": f1score
}
results_df = pd.DataFrame(results, index = [0])

file_path = "/Results_ProposedAlgorithmBERT.csv"
results_file = Path(file_path)
if results_file.is_file():
  read = pd.read_csv(file_path)
  final_df = pd.concat([read, results_df])
  final_df.to_csv(file_path, sep=',', index=False, encoding='utf-8')
else:
  results_df.to_csv(file_path, sep=',', index=False, encoding='utf-8')